In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('tree').getOrCreate()

In [3]:
df = spark.read.csv('College.csv', inferSchema = True, header = True)

In [4]:
df.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [5]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [7]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

In [8]:
assembler = VectorAssembler(inputCols = ['Apps', 'Accept', 'Enroll', 'Top10perc', 
                                         'Top25perc', 'F_Undergrad', 'P_Undergrad', 
                                         'Outstate', 'Room_Board', 'Books', 'Personal',
                                         'PhD', 'Terminal', 'S_F_Ratio', 'perc_alumni',
                                         'Expend', 'Grad_Rate'], outputCol = 'features')

In [9]:
output = assembler.transform(df)

In [10]:
indexer = StringIndexer(inputCol = 'Private', outputCol = 'PrivateIndex')

In [11]:
output_fixed = indexer.fit(output).transform(output)

In [12]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [13]:
final_data = output_fixed.select(['features', 'PrivateIndex'])

In [14]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [15]:
from pyspark.ml.classification import (DecisionTreeClassifier, GBTClassifier,
                                       RandomForestClassifier)

In [16]:
dtc = DecisionTreeClassifier(labelCol = 'PrivateIndex', featuresCol = 'features')
rfc = RandomForestClassifier(labelCol = 'PrivateIndex', featuresCol = 'features',
                             numTrees = 200)
gbt = GBTClassifier(labelCol = 'PrivateIndex', featuresCol = 'features')

In [17]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [18]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [20]:
my_binary_eval = BinaryClassificationEvaluator(labelCol = 'PrivateIndex')

In [21]:
dtc_eval = my_binary_eval.evaluate(dtc_preds)
rfc_eval = my_binary_eval.evaluate(rfc_preds)

In [22]:
print(f"Decision Tree AUC: {dtc_eval}\nRandom Forest AUC: {rfc_eval}")

Decision Tree AUC: 0.9397559991933857
Random Forest AUC: 0.9840693688243597


**since the GBT schema does not raw prediction we need to input rawPredictonCol as 'prediction'**

In [23]:
my_binary_eval2 = BinaryClassificationEvaluator(labelCol = 'PrivateIndex', 
                                                rawPredictionCol = 'prediction')

In [24]:
print(f"Gradient Boosted Tree AUC: {my_binary_eval2.evaluate(gbt_preds)}")

Gradient Boosted Tree AUC: 0.9131881427707199


In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [26]:
acc_eval = MulticlassClassificationEvaluator(labelCol = 'PrivateIndex', metricName='accuracy')

In [27]:
dtc_acc = acc_eval.evaluate(dtc_preds)
rfc_acc = acc_eval.evaluate(rfc_preds)
gbt_acc = acc_eval.evaluate(gbt_preds)

print(f"DTC Accuracy: {dtc_acc}\nRFC Accuracy: {rfc_acc}\nGBT Accuracy: {gbt_acc}")

DTC Accuracy: 0.9082969432314411
RFC Accuracy: 0.9432314410480349
GBT Accuracy: 0.9213973799126638
